# Cartpole Example
This example demonstrates global adversarial stress testing in the classic cartpole environment. The system under test (SUT) is a pre-trained DQN controller with a very low natural rate of failure.

In [ ]:
using Pkg
Pkg.activate(".")

# Loading cart-pole controller (SUT)

In [ ]:
# to learn
using ReinforcementLearning
using ReinforcementLearningExperiments

# to load
using BSON
using Flux
using NNlib
using Random
using ReinforcementLearningCore
using ReinforcementLearningZoo
using StableRNGs

In [ ]:
# This function can be invoked to re-train the DQN policy if desired.
function learn_policy()
    ex = E`JuliaRL_BasicDQN_CartPole`
    ex.stop_condition = StopAfterStep(100_000)
    run(ex)
    learned_policy = ex.policy.policy
    learned_policy.explorer.is_training = false
    bson("dqn_policy.bson", policy = learned_policy)
end

In [ ]:
load_policy() = BSON.load("dqn_policy.bson")[:policy]

# Simulation

In [ ]:
using Random
using Revise
using CommonRLInterface
using AdaStress
import AdaStress.Interface as ASTInterface
using Distributions
using Parameters
using Plots
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear)

In [ ]:
@with_kw mutable struct CartPoleSim <: ASTInterface.GrayBox
    env::ReinforcementLearning.AbstractEnv = CartPoleEnv()
    tmax::Float64 = 100.0
    pi::ReinforcementLearning.AbstractPolicy = load_policy()
    x_dist::ASTInterface.Environment = ASTInterface.Environment(:wind => Normal(0.0, 0.01))
    log::Dict{Symbol, Any} = Dict{Symbol, Any}()
    logging::Bool = false
end

In [ ]:
function ASTInterface.reset!(sim::CartPoleSim)
    ReinforcementLearning.reset!(sim.env)
    if sim.logging
        sim.log[:s] = []
        sim.log[:x] = []
        sim.log[:a] = []
        push!(sim.log[:s], ASTInterface.observe(sim))
    end
end

ASTInterface.environment(sim::CartPoleSim) = sim.x_dist

ASTInterface.observe(sim::CartPoleSim) = copy(ReinforcementLearning.state(sim.env))

function ASTInterface.step!(sim::CartPoleSim, x::ASTInterface.EnvironmentValue)
    a = sim.pi(sim.env)
    s = ReinforcementLearning.state(sim.env)
    s[2] += x[:wind]
    sim.env(a)
    if sim.logging
        push!(sim.log[:s], ASTInterface.observe(sim))
        push!(sim.log[:x], x[:wind])
        push!(sim.log[:a], a)
    end     
end

ASTInterface.isterminal(sim::CartPoleSim) = sim.env.t >= sim.tmax

function ASTInterface.isevent(sim::CartPoleSim)
    state, params = sim.env.state, sim.env.params
    return abs(state[1]) > params.xthreshold || abs(state[3]) > params.thetathreshold
end

interval_dist(x::Real, r::Real) = interval_dist(x, -r, r)

interval_dist(x::Real, l::Real, u::Real) = (l < x < u) ? min(x - l, u - x) : zero(x)

function ASTInterface.distance(sim::CartPoleSim)
    s = ReinforcementLearning.state(sim.env)
    dx = interval_dist(s[1], sim.env.params.xthreshold)
    dθ = interval_dist(s[3], sim.env.params.thetathreshold)
    d = sqrt(dx^2 + dθ^2)
    return d
end

# Trajectory examples

In [ ]:
Random.seed!(0)
p = plot()
sim = CartPoleSim(logging=true)

for _ in 1:100
    ASTInterface.reset!(sim)
    while !(ASTInterface.isterminal(sim) || ASTInterface.isevent(sim))
        x = rand(sim.x_dist; flat=false)
        ASTInterface.step!(sim, x)
    end
    τ = collect(1:length(sim.log[:s]))
    q = getindex.(sim.log[:s], 3)
    plot!(p, τ, q, label=:none, lc=:black, alpha=0.1)
    scatter!(p, [τ[1]], [q[1]], mc=:green, ms=2.0, label=:none)
    scatter!(p, [τ[end]], [q[end]], mc=:red, ms=2.0, label=:none)
end
p

# Failure prevalence

In [ ]:
Random.seed!(0)
c = 0
N = 1_000_000

ASTInterface.reset!(sim)
@showprogress for _ in 1:N
    x = rand(sim.x_dist; flat=false)
    ASTInterface.step!(sim, x)
    d, e = ASTInterface.isterminal(sim), ASTInterface.isevent(sim)
    c += e
    (d || e) && ASTInterface.reset!(sim)
end
rate = c / N

# Failure trajectory example

In [ ]:
Random.seed!(0)
num_eps = 100_000

@showprogress for _ in 1:num_eps
    ASTInterface.reset!(sim)
    while !(ASTInterface.isterminal(sim) || ASTInterface.isevent(sim))
        x = rand(sim.x_dist; flat=false)
        ASTInterface.step!(sim, x)
    end
    ASTInterface.isevent(sim) && break
end
plot(getindex.(sim.log[:s], 3); grid=true, minorgrid=true, ga=0.5)

# AST

In [ ]:
function mdp_env(σ::Float64)
    mdp = ASTInterface.ASTMDP(CartPoleSim(); reward_bonus=100.0)
    mdp.sim.x_dist[:wind] = Normal(0.0, σ)
    return mdp
end

# Single run

In [ ]:
Random.seed!(0)

sac = AdaStress.SoftActorCritic.SAC(;
    obs_dim=4, 
    act_dim=1,
    act_mins=-3.0*ones(1),
    act_maxs=3.0*ones(1),
    hidden_sizes=[30,30,30],
    q_optimizer=AdaStress.SoftActorCritic.AdaBelief(1e-4),
    pi_optimizer=AdaStress.SoftActorCritic.AdaBelief(1e-4),
    alpha_optimizer=AdaStress.SoftActorCritic.AdaBelief(1e-4),
    gamma=1.0,
    num_q=3,
    max_buffer_size=100_000,
    batch_size=1024,
    epochs=100,
    steps_per_epoch=1_000,
    start_steps=10_000,
    max_ep_len=100,
    update_after=10_000,
    update_every=1_000,
    num_batches=100,
    num_test_episodes=100
)

ac, info = sac(() -> mdp_env(0.1))

# Annealing method

In [ ]:
Random.seed!(0)
σs = exp10.(range(-1, -2; length=20))
append!(σs, σs[end])

sac = AdaStress.SoftActorCritic.SAC(;
    obs_dim=4, 
    act_dim=1,
    act_mins=-3.0*ones(1),
    act_maxs=3.0*ones(1),
    hidden_sizes=[30,30,30],
    q_optimizer=AdaStress.SoftActorCritic.AdaBelief(1e-4),
    pi_optimizer=AdaStress.SoftActorCritic.AdaBelief(1e-4),
    alpha_optimizer=AdaStress.SoftActorCritic.AdaBelief(1e-4),
    gamma=1.0,
    num_q=3,
    max_buffer_size=100_000,
    batch_size=1024,
    epochs=100,
    steps_per_epoch=1_000,
    start_steps=10_000,
    max_ep_len=100,
    update_after=10_000,
    update_every=1_000,
    num_batches=1_000, #100,
    num_test_episodes=100,
    displays=[(:fails, mdp->ASTInterface.isevent(mdp.sim)), (:sigma, mdp->NaN)]
)

acs = []
infos = []
@time for (i, σ) in enumerate(σs[1:1])
    sac.displays[end] = (:sigma, mdp->σ)
    sac.update_after = i == 1 ? 10_000 : 0
    sac.start_steps = i == 1 ? 10_000 : 0
    
    ac, info = sac(() -> mdp_env(σ))
    push!.([acs, infos], [ac, info])
end
SoftActorCritic.to_cpu!(sac.buffer)

In [ ]:
plot(vcat(getindex.(infos, "score")...); label=:none)

In [ ]:
plot(vcat(getindex.(infos, "fails")...); label=:none)

In [ ]:
plot(vcat(getindex.(infos, "stdev")...); label=:none)

# Evaluation and buffer analysis

In [ ]:
Random.seed!(0)
mdp = mdp_env(0.1)
N = 100

events = []
returns = []
@showprogress for _ in 1:N
    CommonRLInterface.reset!(mdp)
    r = 0.0
    while !terminated(mdp)
        a = acs[end](Float32.(CommonRLInterface.observe(mdp)), true)
        r += act!(mdp, a)
    end
    push!(events, ASTInterface.isevent(mdp.sim))
    push!(returns, r)
end
@show mean(events)
@show mean(returns)

In [ ]:
buffer = sac.buffer
obs = buffer.obs_buf[:,1:buffer.size]

In [ ]:
k = 10
scatter(obs[1,1:k:end], obs[3,2:k:end]; ms=1.0, msw=0.0, mc=:black, grid=true, gridalpha=0.5)

In [ ]:
histogram2d(obs[1,:], obs[3,:])

In [ ]:
sum(Int.(buffer.done_buf)) / buffer.size

# Network visualizations

In [ ]:
function policy(ac, x, θ)
    o = Float32.([x, 0.0, θ, 0.0])
    a, _ = ac.pi(o, true, false)
    return a[]
end

function value(ac, x, θ)
    o = Float32.([x, 0.0, θ, 0.0])
    a, _ = ac.pi(o, true, false)
    qs = [q.q(vcat(o, a))[] for q in ac.qs]
    return mean(qs)
end

In [ ]:
xs = collect(range(-0.05, 0.05, length=100))
thetas = collect(range(-0.05, 0.05, length=100))

vals = value.(Ref(acs[end]), xs, thetas')'
heatmap(xs, thetas, vals)

In [ ]:
pols = policy.(Ref(acs[end]), xs, thetas')'
heatmap(xs, thetas, pols)

# Sample trajectories pre-annealing

In [ ]:
Random.seed!(0)
pθ = plot(; title="\\theta", ga=0.5, minorgrid=true, minorgridalpha=0.25)
px = plot(; title="x", ga=0.5, minorgrid=true, minorgridalpha=0.25)
mdp = mdp_env(0.1)
mdp.sim.logging = true

for _ in 1:100
    CommonRLInterface.reset!(mdp)
    while !terminated(mdp)
        a = acs[1](Float32.(CommonRLInterface.observe(mdp)), true)
        act!(mdp, a)
    end
    
    τ = collect(1:length(mdp.sim.log[:s]))
    
    θ = getindex.(mdp.sim.log[:s], 3)
    plot!(pθ, τ, θ, label=:none, lc=:black, alpha=0.25)
    scatter!(pθ, [τ[1]], [θ[1]], mc=:green, ms=2.0, label=:none)
    scatter!(pθ, [τ[end]], [θ[end]], mc=:red, ms=2.0, label=:none)
    
    x = getindex.(mdp.sim.log[:s], 1)
    plot!(px, τ, x, label=:none, lc=:black, alpha=0.25)
    scatter!(px, [τ[1]], [x[1]], mc=:green, ms=2.0, label=:none)
    scatter!(px, [τ[end]], [x[end]], mc=:red, ms=2.0, label=:none)
end
display(pθ)
display(px)

In [ ]:
plot(mdp.sim.log[:x]; title="Disturbance", label=:none, lc=:black, ga=0.5, minorgrid=true, minorgridalpha=0.25)

# Sample trajectories post-annealing

In [ ]:
Random.seed!(0)
pθ = plot(; title="\\theta", ga=0.5, minorgrid=true, minorgridalpha=0.25)
px = plot(; title="x", ga=0.5, minorgrid=true, minorgridalpha=0.25)
mdp = mdp_env(0.01)
mdp.sim.logging = true

for _ in 1:100
    CommonRLInterface.reset!(mdp)
    while !terminated(mdp)
        a = acs[end](Float32.(CommonRLInterface.observe(mdp)), true)
        act!(mdp, a)
    end
    
    τ = collect(1:length(mdp.sim.log[:s]))
    
    θ = getindex.(mdp.sim.log[:s], 3)
    plot!(pθ, τ, θ, label=:none, lc=:black, alpha=0.25)
    scatter!(pθ, [τ[1]], [θ[1]], mc=:green, ms=2.0, label=:none)
    scatter!(pθ, [τ[end]], [θ[end]], mc=:red, ms=2.0, label=:none)
    
    x = getindex.(mdp.sim.log[:s], 1)
    plot!(px, τ, x, label=:none, lc=:black, alpha=0.25)
    scatter!(px, [τ[1]], [x[1]], mc=:green, ms=2.0, label=:none)
    scatter!(px, [τ[end]], [x[end]], mc=:red, ms=2.0, label=:none)
end
display(pθ)
display(px)

In [ ]:
plot(mdp.sim.log[:x]; title="Disturbance", label=:none, lc=:black, ga=0.5, minorgrid=true, minorgridalpha=0.25)

# Animation

In [ ]:
function Circle(xc::Real, yc::Real, r::Real; n::Int64=100)
    θ = range(0, 2π; length=n+1)
    xs = @. xc + r*cos(θ)
    ys = @. yc + r*sin(θ)
    return Shape(xs, ys)
end

function Rectangle(xc::Real, yc::Real, l::Real, h::Real)
    xs = [xc - l/2, xc + l/2, xc + l/2, xc - l/2]
    ys = [yc - h/2, yc - h/2, yc + h/2, yc + h/2]
    return Shape(xs, ys)
end

In [ ]:
function movie(sim::CartPoleSim, filename::String="animation")
    cart_length = 0.5
    cart_height = 0.25
    wheel_offset = 0.2
    wheel_height = 0.025
    wheel_radius = 0.05
    pole_length = 1.0

    x_grid = 10
    y_grid = 5
    σ_max = 3
    xlim = sim.env.params.xthreshold * [-1, 1]
    ylim = [0, cart_height + pole_length + 0.1]
    xs = range(xlim..., length=x_grid)
    ys = range(ylim..., length=y_grid)
    X = [x for x in xs for y in ys]
    Y = [y for x in xs for y in ys]
    h = xs[2] - xs[1]

    log = sim.log
    anim = @animate for i in 1:length(log[:s])
        x = log[:s][i][1]
        θ = log[:s][i][3]
        i_trunc = min(i, length(log[:s]) - 1)
        a = log[:a][i_trunc]
        w = log[:x][i_trunc] * 10

        # empty scene
        p = plot(; 
            xlim=xlim, 
            ylim=ylim, 
            yaxis=false, 
            minorticks=10, 
            yticks=false, 
            aspect_ratio=:equal, 
            label=:none
        )

        # wind disturbance
        quiver!(p, X, Y; quiver=(x,y)->[w, 0.0], lc=:dodgerblue)

        # pole
        color = abs(θ) < sim.env.params.thetathreshold ? :blue : :red
        x_pole, y_pole = x, cart_height
        xs = x_pole .+ [0, pole_length] * sin(θ)
        ys = y_pole .+ [0, pole_length] * cos(θ)
        plot!(p, xs, ys; lw=5, lc=color, label=:none)
        scatter!(p, [xs[1]], [ys[1]]; ms=5, mc=color, label=:none)
        scatter!(p, [xs[2]], [ys[2]]; ms=8, mc=color, label=:none)

        # cart body
        color = abs(x) < sim.env.params.xthreshold ? :gray : :red
        plot!(p, Rectangle(x, cart_height/2, cart_length, cart_height); fill=color, label=:none)

        # cart wheels
        plot!(p, Circle(x + wheel_offset, wheel_height, wheel_radius); lw=2, fill=:gray, label=:none)
        plot!(p, Circle(x - wheel_offset, wheel_height, wheel_radius); lw=2, fill=:gray, label=:none)

        # controller force
        dir = a == 2 ? 1 : -1
        xs = (x - dir * cart_length / 2) * [1, 1] .- dir * [0.25, 0]
        ys = cart_height / 2 * [1, 1] 
        plot!(p, xs, ys; arrow=true, lc=:red, lw=4, label=:none)

        p
    end

    return gif(anim, "$filename.mp4", fps = 15)
end

In [ ]:
Random.seed!(0)
p = plot()
mdp = mdp_env(0.01)
mdp.sim.logging = true

CommonRLInterface.reset!(mdp)
while !terminated(mdp)
    a = acs[end](Float32.(CommonRLInterface.observe(mdp)), true)
    act!(mdp, a)
end
movie(mdp.sim)